# Lesson 3c: Summarizing data

> What we have is a data glut.
>
> \- Vernor Vinge, Professor Emeritus of Mathematics, San Diego State University

Computing summary statistics across columns and various groupings within your dataset is a fundamental task in creating insights from your data. This lesson will focus on how we can compute various aggregations with Pandas Series and DataFrames. 

## Learning objectives

By the end of this lesson you will be able to:

- Compute summary statistics across an entire Series
- Compute summary statistics across one or more columns in a DataFrame
- Compute grouped-level summary statistics across one or more columns in a DataFrame

## Simple aggregation

In the previous lesson we learned how to manipulate data across one or more variables within the row(s):

<center>
<img src="https://github.com/bradleyboehmke/uc-bana-6043/blob/main/book/images/series-plus-series.png?raw=true" alt="series-plus-series.png" width="60%" height="60%">
</center>

```{note}
We return the same number of elements that we started with. This is known as a **window function**, but you can also think of it as summarizing at the row-level.
```

We could achieve this result with the following code:

```python
DataFrame['A'] + DataFrame['B']
```

We subset the two Series and then add them together using the `+` operator to achieve the sum. Note that we could also use some other operation on `DataFrame['B']` as long as it returns the same number of elements.

However, sometimes we want to work with data across rows within a variable -- that is, aggregate/summarize values rowwise rather than columnwise.

<center>
<img src="https://github.com/bradleyboehmke/uc-bana-6043/blob/main/book/images/aggregate-series.png?raw=true" alt="aggregate-series.png" width="50%" height="50%">
</center>

```{note}
We return a single value representing some aggregation of the elements we started with. This is known as a **summary function**, but you can think of it as aggregating values across rows.
```

### Summarizing a series

The easiest way to summarize a specific series is by using bracket subsetting notation and the built-in Series methods (i.e. `col.sum()`):

In [1]:
import pandas as pd

ames = pd.read_csv('../data/ames_raw.csv')

ames['SalePrice'].sum()

529732456

Note that a *single value* was returned because this is a **summary operation** -- we are summing the `SalePrice` variable across all rows.

There are other summary methods with a series:

In [2]:
ames['SalePrice'].mean()

180796.0600682594

In [3]:
ames['SalePrice'].median()

160000.0

In [4]:
ames['SalePrice'].std()

79886.69235666493

All of the above methods work on quantitative variables but not character variables. However, there are summary methods that will work on all types of variables:

In [5]:
# Number of unique values in the neighborhood variable
ames['Neighborhood'].nunique()

28

In [6]:
# Most frequent value observed in the neighborhood variable
ames['Neighborhood'].mode()

0    NAmes
Name: Neighborhood, dtype: object

### Knowledge check

```{admonition} Questions:
:class: attention
1. What is the difference between a window operation and a summary operation?
2. What is the mean, median, and standard deviation of the above ground square footage (`Gr Liv Area` variable)?  
3. Find the count of each value observed in the `Neighborhood` column. This may take a Google search. Would you consider the output a summary operation?
```

### Describe method

There is also a method `describe()` that provides a lot of this summary information -- this is especially useful in initial exploratory data analysis.

In [7]:
ames['SalePrice'].describe()

count      2930.000000
mean     180796.060068
std       79886.692357
min       12789.000000
25%      129500.000000
50%      160000.000000
75%      213500.000000
max      755000.000000
Name: SalePrice, dtype: float64

```{note}
The `describe()` method will return different results depending on the `type` of the Series.
```

In [8]:
ames['Neighborhood'].describe()

count      2930
unique       28
top       NAmes
freq        443
Name: Neighborhood, dtype: object

### Summarizing a DataFrame

The above methods and operations are nice, but sometimes we want to work with multiple variables rather than just one. Recall how we select variables from a DataFrame:

* Single-bracket subset notation
* Pass a list of quoted variable names into the list

```python
ames[['SalePrice', 'Gr Liv Area']]
```

We can use *the same summary methods from the Series on the DataFrame* to summarize data:

In [9]:
ames[['SalePrice', 'Gr Liv Area']].mean()

SalePrice      180796.060068
Gr Liv Area      1499.690444
dtype: float64

In [10]:
ames[['SalePrice', 'Gr Liv Area']].median()

SalePrice      160000.0
Gr Liv Area      1442.0
dtype: float64

This returns a `pandas.core.series.Series` object  -- the Index is the variable name and the values are the summarized values.

### The Aggregation method

While summary methods can be convenient, there are a few drawbacks to using them on DataFrames:

1. You can only apply one summary method at a time
2. You have to apply the same summary method to all variables
3. A Series is returned rather than a DataFrame -- this makes it difficult to use the values in our analysis later

In order to get around these problems, the DataFrame has a powerful method `.agg()`:

In [11]:
ames.agg({
    'SalePrice': ['mean']
})

,SalePrice
mean,180796.060068


There are a few things to notice about the `agg()` method:

1. A `dict` is passed to the method with variable names as keys and a list of quoted summaries as values
2. *A DataFrame is returned* with variable names as variables and summaries as rows

```{tip}
The `.agg()` method is just shorthand for `.aggregate()`.
```

In [12]:
# I'm feeling quite verbose today!
ames.aggregate({
    'SalePrice': ['mean']
})

,SalePrice
mean,180796.060068


In [13]:
# I don't have that kind of time!
ames.agg({
    'SalePrice': ['mean']
})

,SalePrice
mean,180796.060068


We can extend this to multiple variables by adding elements to the `dict`:

In [14]:
ames.agg({
    'SalePrice': ['mean'],
    'Gr Liv Area': ['mean']
})

,SalePrice,Gr Liv Area
mean,180796.060068,1499.690444


And because the values of the `dict` are lists, we can do additional aggregations at the same time:

In [15]:
ames.agg({
    'SalePrice': ['mean', 'median'],
    'Gr Liv Area': ['mean', 'min']
})

,SalePrice,Gr Liv Area
mean,180796.060068,1499.690444
median,160000.000000,NaN
min,NaN,334.000000


```{note}
Not all variables have to have the same list of summaries. Note how `NaN` values fill in for those summary statistics _not_ computed on a given variable.
```

### Knowledge check

```{admonition} Questions:
:class: attention
1. Fill in the blanks to compute the average number of rooms above ground (`TotRms AbvGrd`) and the average number of bedrooms above ground (`Bedroom AbvGr`). What type of object is returned?

   ```python
   ames[['______', '______']].______()

2. Use the `.agg()` method to complete the same computation as above. How does the output differ?
3. Fill in the blanks in the below code to calculate the minimum and maximum year built (`Year Built`) and the mean and median number of garage stalls (`Garage Cars`):

   ```python
   ames.agg({
       '_____': ['min', '_____'],
       '_____': ['_____', 'median']
   })

```

### Describe method

While `agg()` is a powerful method, the `describe()` method -- similar to the Series `describe()` method -- is a great choice during exploratory data analysis:

In [16]:
ames.describe()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2930.00000,2.930000e+03,2930.000000,2440.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2907.000000,...,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000
mean,1465.50000,7.144645e+08,57.387372,69.224590,10147.921843,6.094881,5.563140,1971.356314,1984.266553,101.896801,...,93.751877,47.533447,23.011604,2.592491,16.002048,2.243345,50.635154,6.216041,2007.790444,180796.060068
std,845.96247,1.887308e+08,42.638025,23.365335,7880.017759,1.411026,1.111537,30.245361,20.860286,179.112611,...,126.361562,67.483400,64.139059,25.141331,56.087370,35.597181,566.344288,2.714492,1.316613,79886.692357
min,1.00000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,733.25000,5.284770e+08,20.000000,58.000000,7440.250000,5.000000,5.000000,1954.000000,1965.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129500.000000
50%,1465.50000,5.354536e+08,50.000000,68.000000,9436.500000,6.000000,5.000000,1973.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160000.000000
75%,2197.75000,9.071811e+08,70.000000,80.000000,11555.250000,7.000000,6.000000,2001.000000,2004.000000,164.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,213500.000000
max,2930.00000,1.007100e+09,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


```{warning}
What is missing from the above result?
```

The string variables are missing! We can make `describe()` compute on all variable types using the `include` parameter and passing a list of data types to include:

In [17]:
ames.describe(include = ['int', 'float', 'object'])

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
count,2930.00000,2.930000e+03,2930.000000,2930,2440.000000,2930.000000,2930,198,2930,2930,...,2930.000000,13,572,106,2930.000000,2930.000000,2930.000000,2930,2930,2930.000000
unique,NaN,NaN,NaN,7,NaN,NaN,2,2,4,4,...,NaN,4,4,5,NaN,NaN,NaN,10,6,NaN
top,NaN,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,...,NaN,Ex,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,NaN,2273,NaN,NaN,2918,120,1859,2633,...,NaN,4,330,95,NaN,NaN,NaN,2536,2413,NaN
mean,1465.50000,7.144645e+08,57.387372,NaN,69.224590,10147.921843,NaN,NaN,NaN,NaN,...,2.243345,NaN,NaN,NaN,50.635154,6.216041,2007.790444,NaN,NaN,180796.060068
std,845.96247,1.887308e+08,42.638025,NaN,23.365335,7880.017759,NaN,NaN,NaN,NaN,...,35.597181,NaN,NaN,NaN,566.344288,2.714492,1.316613,NaN,NaN,79886.692357
min,1.00000,5.263011e+08,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,12789.000000
25%,733.25000,5.284770e+08,20.000000,NaN,58.000000,7440.250000,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,4.000000,2007.000000,NaN,NaN,129500.000000
50%,1465.50000,5.354536e+08,50.000000,NaN,68.000000,9436.500000,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,160000.000000
75%,2197.75000,9.071811e+08,70.000000,NaN,80.000000,11555.250000,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,213500.000000


## Grouped aggregation

In the section above, we talked about **summary** operations in the context of collapsing a DataFrame to a single row. This is not always the case -- often we are interested in examining specific groups in our data and we want to perform summary operations for these groups. Thus, we are interested in collapsing to a *single row per group*. This is known as a **grouped aggregation**.

For example, in the following illustration we are interested in finding the sum of variable `B` for each category/value in variable `A`.

<center>
<img src="https://github.com/bradleyboehmke/uc-bana-6043/blob/main/book/images/summarizing-by-groups.png?raw=true" alt="summarizing-by-groups.png" width="80%" height="80%">
</center>

This can be useful when we want to aggregate by category:
  * Maximum temperature *by month*
  * Total home runs *by team*
  * Total sales *by neighborhood*
  * Average number of seats *by plane manufacturer*

When we summarize by groups, we can use the same aggregation methods we previously did
  * Summary methods for a specific summary operation: `DataFrame.sum()`
  * Describe method for a collection of summary operations: `DataFrame.describe()`
  * Agg method for flexibility in summary operations: `DataFrame.agg({'VariableName': ['sum', 'mean']})`

The only difference is the need to **set the DataFrame group prior to aggregating**. We can set the DataFrame group by calling the `DataFrame.groupby()` method and passing a variable name:

In [18]:
ames_grp = ames.groupby('Neighborhood')
ames_grp

Notice that a DataFrame doesn't print when it's grouped. The `groupby()` method is just setting the group - you can see the changed DataFrame class:

In [19]:
type(ames_grp)

pandas.core.groupby.generic.DataFrameGroupBy

The groupby object is really just a dictionary of index-mappings, which we could look at if we wanted to:

In [20]:
ames_grp.groups

{'Blmngtn': [52, 53, 468, 469, 470, 471, 472, 473, 1080, 1081, 1082, 1083, 1084, 1741, 1742, 1743, 1744, 2419, 2420, 2421, 2422, 2423, 2424, 2425, 2426, 2427, 2428, 2429], 'Blueste': [298, 299, 932, 933, 934, 935, 1542, 1543, 2225, 2227], 'BrDale': [29, 30, 31, 402, 403, 404, 405, 406, 407, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1675, 1676, 1677, 1678, 1679, 2364, 2365, 2366, 2367, 2368, 2369, 2370, 2371, 2372], 'BrkSide': [129, 130, 191, 192, 193, 194, 195, 196, 197, 198, 199, 614, 615, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 1219, 1220, 1221, 1222, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1901, 1902, 1903, 1904, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2022, 2023, 2024, 2025, 2554, 2555, 2556, 2557, 2670, 2671, 2672, 2673, 2675, 2676, 2677, ...

We can also access a group using the `.get_group()` method:

In [21]:
# get the Bloomington neighborhood group
ames_grp.get_group('Blmngtn').head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
52,53,528228285,120,RL,43.0,3203,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,160000
53,54,528228440,120,RL,43.0,3182,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,192000
468,469,528228290,120,RL,53.0,3684,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,6,2009,WD,Normal,174000
469,470,528228295,120,RL,51.0,3635,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,175900
470,471,528228435,120,RL,43.0,3182,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,192500


If we then call an aggregation method after our `groupby()` call, we will see the DataFrame returned with group-level aggregations:

In [22]:
ames.groupby('Neighborhood').agg({'SalePrice': ['mean', 'median']}).head()

SalePrice          
                       mean    median
Neighborhood                         
Blmngtn       196661.678571  191500.0
Blueste       143590.000000  130500.0
BrDale        105608.333333  106000.0
BrkSide       124756.250000  126750.0
ClearCr       208662.090909  197500.0

This process always follows this model:

<center>
<img src="https://github.com/bradleyboehmke/uc-bana-6043/blob/main/book/images/model-for-grouped-aggs.png?raw=true" alt="model-for-grouped-aggs.png" width="80%" height="80%">
</center>

### Groups as index vs. variables

```{note}
Notice that the grouped variable becomes the Index in our example!
```

In [23]:
ames.groupby('Neighborhood').agg({'SalePrice': ['mean', 'median']}).index

Index(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
       'Crawfor', 'Edwards', 'Gilbert', 'Greens', 'GrnHill', 'IDOTRR',
       'Landmrk', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'NWAmes',
       'NoRidge', 'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW',
       'Somerst', 'StoneBr', 'Timber', 'Veenker'],
      dtype='object', name='Neighborhood')

This is the default behavior of pandas, and probably how pandas wants to be used.  In fact, this is the fastest way to do it, but it's a matter of less than a millisecond. However, you aren't always going to see people group by the index. Instead of setting the group as the index, we can set the group as a variable.

```{tip}
The grouped variable can remain a Series/variable by adding the `as_index = False` parameter/argument to `groupby()`.
```

In [24]:
ames.groupby('Neighborhood', as_index=False).agg({'SalePrice': ['mean', 'median']}).head()

Neighborhood      SalePrice          
                         mean    median
0      Blmngtn  196661.678571  191500.0
1      Blueste  143590.000000  130500.0
2       BrDale  105608.333333  106000.0
3      BrkSide  124756.250000  126750.0
4      ClearCr  208662.090909  197500.0

### Grouping by multiple variables

Sometimes we have multiple categories by which we'd like to group. To extend our example, assume we want to find the average sale price by neighborhood ***AND*** year sold. We can pass a list of variable names to the `groupby()` method:

In [25]:
ames.groupby(['Neighborhood', 'Yr Sold'], as_index=False).agg({'SalePrice': 'mean'})

,Neighborhood,Yr Sold,SalePrice
0,Blmngtn,2006,214424.454545
1,Blmngtn,2007,194671.500000
2,Blmngtn,2008,190714.400000
3,Blmngtn,2009,177266.666667
4,Blmngtn,2010,176000.000000
...,...,...,...
125,Timber,2010,224947.625000
126,Veenker,2006,270000.000000
127,Veenker,2007,253577.777778
128,Veenker,2008,225928.571429


### Knowledge check

```{admonition} Questions:
:class: attention
1. How would you convert the following statement into a grouped aggregation syntax: "what is the average above ground square footage of homes based on neighbhorhood and bedroom count"?
2. Compute the above statement (variable hints: `Gr Liv Area` = above ground square footage, `Neighborhood` = neighborhood, `Bedroom AbvGr` = bedroom count).
3. Using the results from #2, find out which neighborhoods have 1 bedrooms homes that average more than 1500 above ground square feet.
```

## Exercises

```{admonition} Questions:
:class: attention
Using the Ames housing data...
1. What neighbhorhood has the largest median sales price? (hint: check out `sort_values()`)
2. What is the mean and median sales price based on the number of bedrooms (`Bedroom AbvGr`)?
3. Which neighbhorhood has the largest median sales price for 3 bedroom homes? Which neighborhood has the smallest median sales price for 3 bedroom homes?
4. Compute the sales price per square footage (`Gr Liv Area`) per home. Call this variable `price_per_sqft`. Now compute the median `price_per_sqft` per neighborhood. Which neighborhood has the largest median `price_per_sqft`? Does this differ from the neighborhood identified in #1? What information does this provide you?
```

## Computing environment

In [26]:
%load_ext watermark
%watermark -v -p jupyterlab,pandas

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.2.0

jupyterlab: 3.3.2
pandas    : 1.4.2

